In [ ]:
import os

# Set the Hugging Face token
os.environ["HF_HOME"] = "/content/huggingface/cache"
os.environ["HF_TOKEN"] = "HF_TOKEN"
!pip install transformers torch accelerate
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf" # meta-llama/Llama-2-7b-hf

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

from transformers import pipeline

def get_llama_response(prompt: str) -> str:
    """
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        str: The model's response (expected to be 'positive', 'negative', or 'neutral').
    """
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=512,
    )
    # Assuming the response is the sentiment we need, extract and clean it
    response_text = sequences[0]['generated_text']
    print(response_text)
    # Process the response text to find the sentiment word
    sentiment = response_text.split()[-1]  # Assuming the last word is the sentiment
    print("Predicted Sentiment:", sentiment)
    return sentiment

def sentiment_analysis(article: str) -> None:
    """
    Analyzes the sentiment of a given article using the Llama model.

    Parameters:
        article (str): The article text to analyze.

    Returns:
        None: Prints the predicted sentiment of the article.
    """
    positive_example = 'Great Lakes leaders pledge peace. Presidents of the Great Lakes region in Africa have signed a declaration designed to bring peace to the region.'
    negative_example = """'Ballesteros Under Tour Probe for Alleged Assault. SOTOGRANDE, Spain (Reuters) - Severiano Ballesteros is to be investigated by the European Tour following an alleged assault on one of the tour's tournament directors at his home course Pedrena."""
    prompt = f"This is an example of an article with a positive sentiment: {positive_example} This is an example of an article with negative sentiment: {negative_example} Given this, predict the sentiment of the following article as positive, negative, or neutral in only one word: {article}\n"
    get_llama_response(prompt)

    # article_2= """Great Lakes leaders pledge peace Presidents of the Great Lakes region in Africa have signed a declaration designed bring peace to the region. """
    # prompt_2 = f"""
    # Act as a sentiment analyzer for a media outlet. This is an example of {positive_example} article with a positive sentiment. This is an example of {negative_example} of article with negative sentiment. Classify <example> as positive, negative or neutral.
    # {article_2}
    # """
    # get_llama_response(prompt_2)

# Example usage
template = """
Please generate an answer to the following question using any relevant information from the provided context:

Context: {context}

Question: {question}

Helpful Answer:
"""

custom_rag_prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
)


article = "Inflation fears, oil swings to keep sentiment low MUMBAI: The market is expected to remain volatile ahead of the expiry of futures contract on Thursday. The mood will remain subdued amid worries over a fresh rise in global crude oil prices and inflation in the country, say brokers."
sentiment_analysis(article)


prompt = """
Give 50 article headlines about the Israel-Palestine war along with their source
"""
response = get_llama_response(prompt)
print(response)


def sentiment_analysis(article):
  positive_article = 'Great Lakes leaders pledge peace Presidents of the Great Lakes region in Africa have signed a declaration designed bring peace to the region'
  negative_article = """'Ballesteros Under Tour Probe for Alleged Assault SOTOGRANDE, Spain (Reuters) - Severiano Ballesteros is to be investigated by the European Tour following an alleged assault on one of the tour's tournament directors at his home course Pedrena"""
  # article = ''
  # prompt = f"""What is the sentiment of  {article} article. Respond with either positive, negative, or neutral."""
  # get_llama_response(prompt)
  # article = ''
  prompt = f"""This is an example of {positive_article} article with a positive sentiment. This is a example of {negative_article} of article
  with negative sentiment Given this, predict the sentiment of the following article as positive, negative, or neutral.
  {article}
  """
  get_llama_response(prompt)
  # prompt = f"""
  # Act as a sentiment analyzer for a media outlet. This is an example of {positive_article} article with a positive sentiment. This is an example of {negative_article} of article with negative sentiment. Classify <example> as positive, negative or neutral.
  # {article}
  # """
  # get_llama_response(prompt)